In [2]:
from __future__ import print_function

# Standard Py Imports
import os
import json
import pickle
import os.path
import requests

# Google Imports
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [3]:
# First, set the following elements as environment variables ony our machine
# ** DISCLAIMER **: if you push these to GitHub, I will be sad.
# ** FOR MACOSX: After you source, .bash_profile, might need full system restart to discover env vars. 
project_id = os.environ["goog_project_id"]
client_id = os.environ["goog_client_id"]
client_secret = os.environ["goog_client_secret"]

In [4]:
r_url = "http://127.0.0.1:5000/process_bert"

In [53]:
# Add the secret stuff to our credentials document...
with open("config.json", "r") as f:
    creds = json.load(f)
print(creds)
creds['web']['client_id'] = client_id
creds['web']['client_secret'] = client_secret
creds['web']['project_id'] = project_id

# And write it.
with open("credentials.json", "w") as f: 
    json.dump(creds, f)

{'web': {'auth_uri': 'https://accounts.google.com/o/oauth2/auth', 'token_uri': 'https://oauth2.googleapis.com/token', 'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs', 'redirect_uris': ['http://localhost', 'http://localhost/potato', 'http://localhost:8080/Callback'], 'javascript_origins': ['http://localhost:8080']}}


In [54]:
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly', 'https://www.googleapis.com/auth/drive.readonly']

# Stolen from Google Quickstart docs
# https://developers.google.com/drive/api/v3/quickstart/python
def do_login_flow():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return creds, None  # Returning None because Tyler can't figure out how he wants to structure this yet. 

# THIS should force-open a Google Auth window in your local browser. If not, you can manually copy-paste it. 
auth_creds = do_login_flow()

# Now delete the file so you don't accidentally `git add` it. 
os.remove("credentials.json")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=364500245041-r1eebsermd1qp1qo68a3qp09hhpc5dfi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57355%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=U8sqbdtW7T0YZZlbNsojguaBqbOj6r&access_type=offline


In [55]:
file_of_choice = "1nuPQE15_5inIe6exuqaZ_Qd7DjBEJRBsIXOYKjURSRY"  # this is Will-Tyler Xtract document

r = requests.post(url=r_url,
                  data=pickle.dumps({'auth_creds': auth_creds, 
                                     'repo_type': 'GDRIVE', 
                                     'is_gdoc': True,
                                     'extension': None,
                                     'file_id': file_of_choice}) )

print(r.content)
print(pickle.loads(r.content))

b'\x80\x03}q\x00X\x03\x00\x00\x00w2vq\x01}q\x02(X\x04\x00\x00\x00pathq\x03X1\x00\x00\x001nuPQE15_5inIe6exuqaZ_Qd7DjBEJRBsIXOYKjURSRY..txtq\x04X\x05\x00\x00\x00modelq\x05h\x01X\x05\x00\x00\x00errorq\x06\x89X\x0c\x00\x00\x00error_reasonq\x07X\x00\x00\x00\x00q\x08X\x03\x00\x00\x00repq\tcnumpy.core.multiarray\n_reconstruct\nq\ncnumpy\nndarray\nq\x0bK\x00\x85q\x0cC\x01bq\r\x87q\x0eRq\x0f(K\x01M,\x01\x85q\x10cnumpy\ndtype\nq\x11X\x02\x00\x00\x00f4q\x12K\x00K\x01\x87q\x13Rq\x14(K\x03X\x01\x00\x00\x00<q\x15NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x16b\x89B\xb0\x04\x00\x00\xa8\xbf\x05<[\x83B<\xe5\x06?\xbb\xedj\x08=M\x8c\xf7\xbc\xcf\x9dr\xbb\x93\xb4\\<\xc8S\xbb\xbcw\xc9\xb5<\xa1\xc8\xae<\xb8\xee\xc0\xbc\x84\xd9\xac\xbc\xe4\xfbN\xbc\xe1\xee\x8e<\xe3\xe5\x1d\xbd\x96\xe0\xa0<7\xach<sN\x1f=\x8e\x89!\xbc\xaf\xcd%\xbd3\x9c\xcd\xbc\x04\x9e\x94;\xea\xb1`\xbc\x9c\xb9*<4@\xad\xbb\xc0f\x9e\xbc\xf1\x16\xf6\xbc\\\xc0\xf9<\x86\xa9\x92<\xe8<\xf2\xbbZj\xea\xbc\xff\xe4\x1c\xbcP\xed\x1b\xbc\x8c\x12\xce\xbc\x8